In [1]:
# Fungsi untuk mengenkripsi teks

def encrypt_aes_256(key, plaintext):
    # Buat salt dan IV (Initialization Vector)
    salt = os.urandom(16)  # Pembuatan salt untuk derivasi kunci
    iv = os.urandom(16)    # Pembuatan IV untuk mode CBC

    # KDF (Key Derivation Function) untuk menghasilkan kunci AES-256 dari key
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    aes_key = kdf.derive(key.encode())  # Derivasi kunci (Tahap awal untuk AddRoundKey)

    # Padding plaintext agar sesuai dengan blok AES (16 bytes)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    # Enkripsi
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Proses berikut secara implisit mencakup tahapan AES:
    # 1. SubBytes: Substitusi byte dilakukan dalam `encryptor.update()` untuk memperkenalkan kebingungan pada data.
    # 2. ShiftRows: Perpindahan siklik byte diimplementasikan dalam proses internal AES di `encryptor.update()`.
    # 3. MixColumns: Transformasi linier pada kolom data juga dilakukan di `encryptor.update()`.
    # 4. AddRoundKey: XOR antara data dengan kunci ronde dilakukan di setiap ronde enkripsi.
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return salt, iv, ciphertext

# Fungsi untuk mendekripsi teks
def decrypt_aes_256(key, salt, iv, ciphertext):
    # KDF untuk menghasilkan kunci AES-256 dari key
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    aes_key = kdf.derive(key.encode())

    # Dekripsi
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Unpadding untuk mengembalikan teks asli
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_data) + unpadder.finalize()

    return plaintext.decode()

# Contoh penggunaan
if __name__ == "__main__":
    # Teks asli
    plaintext = "UNIVERSITAS ESA UNGGUL ADALAH PERGURUAN TINGGI TERAKREDITASI UNGGUL"
    # Key untuk enkripsi (harus panjang yang memadai)
    key = "supersecretpassword1234567890"  # Minimal 32 karakter untuk keamanan optimal

    # Enkripsi
    salt, iv, ciphertext = encrypt_aes_256(key, plaintext)
    print(f"Ciphertext: {ciphertext.hex()}")

    # Dekripsi
    decrypted_text = decrypt_aes_256(key, salt, iv, ciphertext)
    print(f"Decrypted text: {decrypted_text}")


ModuleNotFoundError: No module named 'cryptography'